In [3]:
import os
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pymongo
import json

## Connect mongoDB in EC2 

In [ ]:
connection = pymongo.MongoClient('mongodb://{}:{}@13.125.31.93'.format(username, password))
db = connection.app_review

In [128]:
BASEURL = 'https://play.google.com/store/apps/category/'
category = ['FINANCE', 'SHOPPING', 'SPORTS', 'VIDEO_PLAYERS',
            'MEDICAL', 'TRAVEL_AND_LOCAL', 'NEWS_AND_MAGEZINES', 
            'COMICS', 'DATING', 'BOOKS_AND_REFERENCE', 'BEAUTY',
           'AUTO_AND_VEHICLES', 'PARENTING']
ENDURL = '/collection/topselling_free'

GROSSING = 'https://play.google.com/store/apps/collection/topgrossing'
FREE = 'https://play.google.com/store/apps/collection/topselling_free'
PAID = 'https://play.google.com/store/apps/collection/topselling_paid'

target_url = list()
for item in category:
    target_url.append(BASEURL + str(item) + ENDURL)
    
target_url.append(GROSSING)
target_url.append(FREE)
target_url.append(PAID)

for url in target_url:
    print(url)

https://play.google.com/store/apps/category/FINANCE/collection/topselling_free
https://play.google.com/store/apps/category/SHOPPING/collection/topselling_free
https://play.google.com/store/apps/category/SPORTS/collection/topselling_free
https://play.google.com/store/apps/category/VIDEO_PLAYERS/collection/topselling_free
https://play.google.com/store/apps/category/MEDICAL/collection/topselling_free
https://play.google.com/store/apps/category/TRAVEL_AND_LOCAL/collection/topselling_free
https://play.google.com/store/apps/category/NEWS_AND_MAGEZINES/collection/topselling_free
https://play.google.com/store/apps/category/COMICS/collection/topselling_free
https://play.google.com/store/apps/category/DATING/collection/topselling_free
https://play.google.com/store/apps/category/BOOKS_AND_REFERENCE/collection/topselling_free
https://play.google.com/store/apps/category/BEAUTY/collection/topselling_free
https://play.google.com/store/apps/category/AUTO_AND_VEHICLES/collection/topselling_free
https:/

In [85]:
def scrape_to_db(url):
    
    connection = pymongo.MongoClient('mongodb://{}:{}@13.125.31.93'.format('oniun', 'wkatlf2ehd'))
    db = connection.app_review
    collection = db.reviews
    
    # PhantomJS driver실행
    driver = webdriver.PhantomJS('PhantomJS')

    # google play page를 띄움
    driver.get(url)

    SCROLL_PAUSE_TIME = 3
    # Get Scroll Height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        #scroll down to botton
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        #wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        #calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

        page_source = driver.page_source

        soup = BeautifulSoup(page_source, 'html.parser')
        rank = soup.select('div.id-card-list.card-list.two-cards > div > div > div.details > a.title')
    
    try:
    
        for num, item in enumerate(rank):
            adders = str(item).split()[3]
            adder = re.search('/[\w/?=.]+', adders)

            add_url = adder.group()

            # app name
            name = re.search('(?<=...)[\w ]+', str(item.get_text()).strip(" "))
            get_app = name.group()

            res2 = requests.get('http://play.google.com' + str(add_url))
            soup2 = BeautifulSoup(res2.content, 'html.parser')

            users = soup2.select('div.review-info > span.author-name')
            dates = soup2.select('div.review-info > span.review-date')
            rates = soup2.select('div.review-info > div.review-info-star-rating')
            reviews = soup2.select('div > div.review-body.with-review-wrapper')

            for user, date, review, rate in zip(users, dates, reviews, rates):

                get_user = user.get_text()
                get_date = date.get_text()

                m = re.search('\d\w\w', str(rate))
                m2 = re.search('\d', m.group())

                get_rate = m2.group()
                get_review = review.get_text().strip(" ").rstrip('전체 리뷰')              

                post = {
                    "app": get_app, "date": get_date, "rate": get_rate,
                    "review": get_review, "user": get_user

                }

                collection.insert_one(post)
                
    except:
        print("Something Wrong")
            
    
    driver.close()
    driver.quit()
    connection.close()
    
    return "Mission Completed"

In [87]:
%%time
scrape_to_db('https://play.google.com/store/apps/category/PARENTING/collection/topselling_free')

Something Wrong
CPU times: user 5min 15s, sys: 2.18 s, total: 5min 17s
Wall time: 11min 54s


'Mission Completed'

In [86]:
%%time
scrape_to_db('https://play.google.com/store/apps/category/AUTO_AND_VEHICLES/collection/topselling_free')

CPU times: user 6min 56s, sys: 3.44 s, total: 7min
Wall time: 16min 23s


'Mission Completed'

In [62]:
%%time
scrape_to_db('https://play.google.com/store/apps/category/BEAUTY/collection/topselling_free')

Something Wrong
Something Wrong
Something Wrong
CPU times: user 6min 54s, sys: 2.71 s, total: 6min 56s
Wall time: 12min 7s


'Mission Completed'

In [61]:
%%time
scrape_to_db('https://play.google.com/store/apps/category/BOOKS_AND_REFERENCE/collection/topselling_free')

CPU times: user 8min 28s, sys: 3.66 s, total: 8min 32s
Wall time: 15min 38s


'Mission Completed'

In [60]:
%%time
scrape_to_db('https://play.google.com/store/apps/category/DATING/collection/topselling_free')

Something Wrong
Something Wrong
Something Wrong
Something Wrong
Something Wrong
Something Wrong
Something Wrong
Something Wrong
CPU times: user 6min 26s, sys: 2.13 s, total: 6min 28s
Wall time: 11min 56s


'Mission Completed'

In [38]:
scrape_to_db('https://play.google.com/store/apps/category/COMICS/collection/topselling_free')

Something Wrong


'Mission Completed'

In [ ]:
import time

In [18]:
%%time
for num, item in enumerate(target_url):
    scrape_to_db(item)
    print(num,"th scraping has been finished.")
    print("Take a break for a while, then do the next scraping")
    time.sleep(5)

UnboundLocalError: local variable 'rank' referenced before assignment

### CSV file insert into mongoDB

In [ ]:
def csv_to col(username, password):
    connection = pymongo.MongoClient('mongodb://{}:{}@13.125.31.93'.format(username, passowrd))
    db = connection.app_review 
    collection = db.reviews

    data = pd.read_csv('dataset.csv')
    data_json = json.loads(data.to_json(orient='records'))
    collection.insert_many(data_json)

    connection.close()
    return "dump completed"

csv_to_col('oniun', 'wkatlf2ehd')

### mongoDB collection to pandas DataFrame

In [92]:
import pandas as pd
from pymongo import MongoClient

def col_to_df(username, password):
    
    # make connection to mongodb
    connection = pymongo.MongoClient('mongodb://{}:{}@13.125.31.93'.format(username, password))
    db = connection.app_review
    review = pd.DataFrame(list(db.reviews.find()))

    connection.close()
    print("Now you can use mongoDB collection as DataFrame")

    # preprocessing: drop_duplication, etc
    review = review.drop_duplicates(subset=['app', 'user', 'review', 'date', 'rate'], keep='first')
    review = review.drop_duplicates(subset=['review'], keep='first')
    review = review.set_index('app')
    review = review.drop(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'])
    review = review.reset_index()
    review['rate'] = review['rate'].replace(['0'], ['1'])

    # store review as csv file
    review.to_csv('data/dataset.csv', sep=',', encoding='utf-8', index=False)
    
    return review.info()

Now you can use mongoDB collection as DataFrame
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102692 entries, 0 to 102691
Data columns (total 6 columns):
app       102692 non-null object
_id       102692 non-null object
date      102692 non-null object
rate      102692 non-null object
review    102692 non-null object
user      102692 non-null object
dtypes: object(6)
memory usage: 4.7+ MB


In [93]:
col_to_df('oniun', 'wkatlf2ehd')
review = pd.read_csv('data/dataset.csv')

In [94]:
review['review'].nunique()

102692

In [95]:
review['app'].nunique()

3725

In [100]:
review.isnull().sum()


app       0
_id       0
date      0
rate      0
review    0
user      0
dtype: int64

In [97]:
review.dtypes

app       object
_id       object
date      object
rate       int64
review    object
user      object
dtype: object

In [101]:
review.shape

(102692, 6)

In [102]:
review.tail()

,app,_id,date,rate,review,user
102687,SmiRing,5a2ab22e8abf5a3f6f3b1138,2012년 12월 10일,5,신기해요 와~신기해요...근대 어떻게 이런 평범한 소릴듣고 잠을 자지?,정윤지
102688,SmiRing,5a2ab22e8abf5a3f6f3b1139,2015년 8월 26일,1,앙 전혀 효과 못봄 아기가 들을수록 난감하게 울어대요 이걸 막 던져가주구 이걸 지웟어용,리릴
102689,SmiRing,5a2ab22e8abf5a3f6f3b113a,2013년 1월 19일,5,수중+고동이젤조아요잘때 잠자기조은소,이효배
102690,마미새미,5a2ab23d8abf5a3f6f3b113b,2017년 5월 5일,5,무료수업 듣고싶어요,붐3 Boom3
102691,마미새미,5a2ab23d8abf5a3f6f3b113d,2017년 9월 1일,5,진석이 코 개큼,영채


In [103]:
review.groupby('app').count()

,_id,date,rate,review,user
app,,,,,
,752,752,752,752,752
010대리운전,1,1,1,1,1
1004서비스,3,3,3,3,3
10타줄이는 골프,5,5,5,5,5
119퀵서비스,7,7,7,7,7
11번가 쇼킹딜,38,38,38,38,38
15885585 행복한길대리운전,5,5,5,5,5
1800,4,4,4,4,4
1Q MTS,42,42,42,42,42
